In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from pipeline import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%env "WANDB_NOTEBOOK_NAME" "mete-nn.ipynb"

env: "WANDB_NOTEBOOK_NAME"="mete-nn.ipynb"


In [6]:
pipeline = AABB245_Pipeline(num_blocks=8, extrema_th=8)
dataset_24 = PolymerDataset(['../data/AA66266AA.npy', '../data/AA66466AA.npy'], pipeline)

Processing ../data/AA66466AA.npy: 100%|██████████| 20752/20752 [01:59<00:00, 173.55it/s]


In [7]:
dataset_24.data.shape, dataset_24.labels.shape

(torch.Size([40837, 8, 19]), torch.Size([40837]))

In [8]:
train_data_24, test_data_24 = train_test_split(dataset_24)

In [9]:
model_24 = MultiOutputLSTM(input_dim=dataset_24.num_features, output_dim=dataset_24.num_classes, num_blocks=dataset_24.num_blocks)

In [10]:
train(train_dataset=train_data_24, test_dataset=test_data_24, model=model_24, batch_size=512, num_epochs=100, lr_rate=0.001)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mismayil (use `wandb login --relogin` to force relogin)


epoch=0/100, loss=0.6764074340462685, accuracy=59.62350845336914
epoch=1/100, loss=0.6192255327478051, accuracy=66.80440521240234
epoch=2/100, loss=0.5654871677979827, accuracy=72.36302185058594
epoch=3/100, loss=0.532415000256151, accuracy=75.2586441040039
epoch=4/100, loss=0.5085938037373126, accuracy=76.88399505615234
epoch=5/100, loss=0.49382772156968713, accuracy=77.65228271484375
epoch=6/100, loss=0.48406263208016753, accuracy=78.05632019042969
epoch=7/100, loss=0.47706004884094, accuracy=78.35016632080078
epoch=8/100, loss=0.4715539109893143, accuracy=78.60728454589844
epoch=9/100, loss=0.46649290062487125, accuracy=78.8980712890625
epoch=10/100, loss=0.46206985227763653, accuracy=79.03887176513672
epoch=11/100, loss=0.45810271240770817, accuracy=79.25926208496094
epoch=12/100, loss=0.45477125933393836, accuracy=79.37251281738281
epoch=13/100, loss=0.4504957995377481, accuracy=79.60820007324219
epoch=14/100, loss=0.4461863008327782, accuracy=79.66329956054688
epoch=15/100, loss=

test_accuracy,▁
test_f1 score,▁
test_precision,▁
test_recall,▁
train_accuracy,▁▅▇▇▇▇▇▇▇▇▇▇████████████████████████████
train_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,0.82258
test_f1 score,0.82131
test_precision,0.83396
test_recall,0.80904
train_accuracy,82.50076


MultiOutputLSTM(
  (lstm): LSTM(19, 1, batch_first=True)
  (linear1): Linear(in_features=8, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=2, bias=True)
)